In [1]:
import numpy as np

from sklearn.covariance import GraphLasso, GraphLassoCV

# Graph inference

## Graph Lasso (fixed point in time, $t$)

In [2]:
import numpy as np
def sparseinverse_cov(d):
    """Generate a covariance matrix with Sparse Inverse method.
    Sparse graph where each element depends only on a subset of others."""
    # W is a weight symmetric matrix.
    # It has -1 if two vertices are connected, 0 otherwise.
    # The diagonal is the sum of rows (or columns).
    W = np.zeros((d, d))
    for i in range(d):
        W[i, i+1:] = np.random.randint(-1, 1, d-i-1)
    W += W.T
    w = [abs(np.sum(vec)) for vec in W]
    laplacian = W + np.diag(w)
    return np.linalg.inv(laplacian + np.random.randn() ** 2 * np.eye(d))

In [3]:
theta_t = sparseinverse_cov(3)

In [4]:
theta_t

array([[ 1.44215928,  0.68926196,  0.91547937],
       [ 0.68926196,  1.44215928,  0.91547937],
       [ 0.91547937,  0.91547937,  1.21594187]])

In [5]:
from sklearn.covariance import GraphLasso, GraphLassoCV

In [6]:
def random_covariance(n):
    Sinv = np.eye(n)
    idx = np.random.randint(2, size=n*n).reshape(n,n);
    for i in range(n):
        for j in range(n):
            Sinv[i,j] = idx[i,j]
    Sinv = Sinv + Sinv.T
    if np.min(np.linalg.eigh(Sinv)[0]) < 0:
        Sinv = Sinv + 1.1*np.abs(np.min(np.linalg.eigh(Sinv)[0]))*np.eye(n);

    S = np.linalg.inv(Sinv);
    return S, Sinv

In [7]:
n_dim = 10
n_samples = 50
alpha=2*np.sqrt(np.log(n_dim) / n_samples)
print "alpha:", alpha

# true_covariance = sparseinverse_cov(n_dim)
# true_covariance /= np.diag(true_covariance)[0]
true_covariance, Sinv = random_covariance(n_dim)
X = np.random.multivariate_normal(np.zeros(n_dim), true_covariance, n_samples)

alpha: 0.429193205258


In [33]:
gl = GraphLasso(mode='cd', alpha=alpha, verbose=0, max_iter=200)
gl = GraphLassoCV(verbose=0, max_iter=500, alphas=20)
gl.fit(X)

GraphLassoCV(alphas=20, assume_centered=False, cv=None, enet_tol=0.0001,
       max_iter=500, mode='cd', n_jobs=1, n_refinements=4, tol=0.0001,
       verbose=0)

In [26]:
from rgi import admm_covariance; reload(admm_covariance)

Cov, Z, hist = admm_covariance.covsel(X, .05,rho=2,alpha=alpha, verbose=0)

# print gl.error_norm(true_covariance)
# print np.linalg.norm(gl.covariance_ - true_covariance)

In [27]:
print(np.linalg.norm(Sinv - Cov))

12.0589140965


In [34]:
print(np.linalg.norm(gl.precision_ - Sinv))

7.33373554946


In [29]:
Z

array([[ 5.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0.4,  0. ,  0. ,  0. ],
       [ 0. ,  4. ,  0.7, -0.3,  0. , -0.3,  0.9,  0. ,  0.4,  0. ],
       [ 0. ,  0.7,  3.2,  1. ,  0. ,  1.2,  0. ,  0. ,  0. ,  0. ],
       [ 0. , -0.3,  1. ,  2.6,  1.1, -0.1,  0.9,  0. , -0. ,  0. ],
       [ 0. ,  0. ,  0. ,  1.1,  4. ,  0.7,  0. ,  0. ,  0. , -0.1],
       [ 0. , -0.3,  1.2, -0.1,  0.7,  3.1,  0.4,  0. ,  0.3,  0.1],
       [ 0.4,  0.9,  0. ,  0.9,  0. ,  0.4,  2.6,  0. , -0.1, -0.3],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3.7,  0. ,  0. ],
       [ 0. ,  0.4,  0. , -0. ,  0. ,  0.3, -0.1,  0. ,  4. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -0.1,  0.1, -0.3,  0. ,  0. ,  4.5]])

In [30]:
np.set_printoptions(precision=1, suppress=True)

In [35]:
gl.precision_

array([[ 8.8,  0. , -0.6, -0.9, -0. ,  1.3,  1.2,  1.2,  1.4,  0. ],
       [ 0. ,  6.6,  1.7, -0.4, -0. , -0. ,  1.7, -0.1,  1.5,  0.3],
       [-0.6,  1.7,  5.4,  1.9, -0. ,  2.4,  0.5,  0. ,  0. ,  0.8],
       [-0.9, -0.4,  1.9,  4. ,  1.8, -0. ,  1.5,  0.2, -1. ,  0.1],
       [-0. , -0. , -0. ,  1.8,  6.7,  1.7,  0. ,  0.6,  1.4, -0.9],
       [ 1.3, -0. ,  2.4, -0. ,  1.7,  5.3,  0.7, -0. ,  1.7,  0.4],
       [ 1.2,  1.7,  0.5,  1.5,  0. ,  0.7,  4. ,  0.1, -0.4, -0.5],
       [ 1.2, -0.1,  0. ,  0.2,  0.6, -0. ,  0.1,  4.9, -0. ,  0.6],
       [ 1.4,  1.5,  0. , -1. ,  1.4,  1.7, -0.4, -0. ,  6.6, -0.8],
       [ 0. ,  0.3,  0.8,  0.1, -0.9,  0.4, -0.5,  0.6, -0.8,  6.7]])

In [31]:
Sinv

array([[ 6.5,  1. ,  0. ,  0. ,  0. ,  1. ,  2. ,  2. ,  1. ,  1. ],
       [ 1. ,  6.5,  2. ,  0. ,  1. ,  1. ,  2. ,  0. ,  1. ,  1. ],
       [ 0. ,  2. ,  6.5,  2. ,  0. ,  2. ,  0. ,  1. ,  1. ,  2. ],
       [ 0. ,  0. ,  2. ,  4.5,  2. ,  0. ,  2. ,  1. ,  0. ,  1. ],
       [ 0. ,  1. ,  0. ,  2. ,  6.5,  2. ,  1. ,  1. ,  2. ,  0. ],
       [ 1. ,  1. ,  2. ,  0. ,  2. ,  4.5,  1. ,  1. ,  2. ,  1. ],
       [ 2. ,  2. ,  0. ,  2. ,  1. ,  1. ,  4.5,  1. ,  0. ,  0. ],
       [ 2. ,  0. ,  1. ,  1. ,  1. ,  1. ,  1. ,  6.5,  0. ,  2. ],
       [ 1. ,  1. ,  1. ,  0. ,  2. ,  2. ,  0. ,  0. ,  6.5,  0. ],
       [ 1. ,  1. ,  2. ,  1. ,  0. ,  1. ,  0. ,  2. ,  0. ,  6.5]])

In [32]:
Cov

array([[ 5.1,  0. , -0. ,  0. , -0. ,  0. ,  0.4,  0. , -0. , -0. ],
       [ 0. ,  4. ,  0.7, -0.3, -0. , -0.3,  0.9,  0. ,  0.4, -0. ],
       [-0. ,  0.7,  3.2,  1. ,  0. ,  1.2,  0. , -0. ,  0. ,  0. ],
       [ 0. , -0.3,  1. ,  2.6,  1.1, -0.1,  0.9,  0. , -0. ,  0. ],
       [-0. , -0. ,  0. ,  1.1,  4. ,  0.7,  0. ,  0. ,  0. , -0.1],
       [ 0. , -0.3,  1.2, -0.1,  0.7,  3.1,  0.4,  0. ,  0.3,  0.1],
       [ 0.4,  0.9,  0. ,  0.9,  0. ,  0.4,  2.6, -0. , -0.1, -0.3],
       [ 0. ,  0. , -0. ,  0. ,  0. ,  0. , -0. ,  3.7, -0. , -0. ],
       [-0. ,  0.4,  0. , -0. ,  0. ,  0.3, -0.1, -0. ,  4. ,  0. ],
       [-0. , -0. ,  0. ,  0. , -0.1,  0.1, -0.3, -0. ,  0. ,  4.5]])

In [28]:
import pandas as pd
import numpy as np
from sklearn.covariance import GraphLasso, GraphLassoCV, empirical_covariance

df_x = pd.read_csv("/home/fede/projects_local/kdvs/", delimiter='\t', comment='#')
df_y = pd.read_csv("/home/fede/projects_local/petretto/data/fmf_labels.txt", delimiter='\t')

X = df_x.values
y = df_y.values.ravel()

In [33]:
df_x.loc[:, df_x.columns.str.startswith("FMF_C")].values

array([[ 25.69016,  25.48889,  25.51938, ...,  26.62533,  25.77399,
         25.82825],
       [ 20.96733,  21.09571,  16.70307, ...,  24.32992,  23.06976,
         22.20353],
       [ 25.31748,  25.55324,  25.10913, ...,  24.79872,  25.27302,
         25.17329],
       ..., 
       [ 20.97036,  19.91314,  20.00703, ...,  17.62372,  20.86982,
         20.09159],
       [ 21.64689,  21.6591 ,  22.15527, ...,  21.91561,  22.12682,
         22.4864 ],
       [ 22.84217,  23.94699,  23.8261 , ...,  23.58451,  23.27881,
         22.5633 ]])

In [18]:
emp_cov = empirical_covariance(X[y=='A'], assume_centered=False)

In [37]:
gl = GraphLasso(verbose=1, max_iter=200)
gl.fit(df_x.loc[:, df_x.columns.str.startswith("FMF_C")].values.T)

[graph_lasso] Iteration   0, cost  6.99e+03, dual gap -6.170e+03
[graph_lasso] Iteration   1, cost  nan, dual gap nan


FloatingPointError: Non SPD result: the system is too ill-conditioned for this solver. The system is too ill-conditioned for this solver

In [28]:
n, d = 400, 10

X = np.random.randn(n, d)
beta = np.ones(d); beta[:d//2] = 0

y = X.dot(beta) + np.random.randn(n) * 0.1

In [29]:
beta

array([ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.])

In [30]:
from sklearn.linear_model import Lasso

beta_lasso = Lasso(alpha=0.1).fit(X, y).coef_

In [31]:
beta_lasso

array([ 0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.87164806,  0.87821874,  0.90753309,  0.90732306,  0.88892157])

In [32]:
from rgi import admm_lasso as lasso; reload(lasso)

z = lasso.lasso(X, y, lamda=0.1, rho=1, alpha=1)

In [33]:
z

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.99724445,  1.0013197 ,  1.00083042,  1.00506278,  0.995881  ])

In [34]:
from rgi import admm_group_lasso as grouplasso; reload(grouplasso)

z_group = grouplasso.group_lasso(X, y, lamda=0.1, p=[[i] for i in range(10)],
                                 rho=1, alpha=1)

In [35]:
z_group

array([-0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.99692236,  1.00100604,  1.00058324,  1.00481119,  0.99559972])

In [36]:
print "Error skle: %.3f" % np.linalg.norm(beta_lasso - beta)
print "Error admm: %.3f" % np.linalg.norm(z - beta)
print "Error admm_group: %.3f" % np.linalg.norm(z_group - beta)

Error skle: 0.247
Error admm: 0.007
Error admm_group: 0.007


In [42]:
from rgi import admm_group_lasso_overlap as grouplassooverlap; reload(grouplassooverlap)

z_group = grouplassooverlap.GroupLassoOverlap(
        alpha=2, rho=2, groups=[[0,1,2,3,4], [3,4,5], [5,6,7,8,9]], verbose=0, max_iter=1000, tol=1e-6).fit(
X, y).coef_

In [43]:
print "Error admm_group_overlap: %.3f" % np.linalg.norm(z_group - beta)

Error admm_group_overlap: 0.016


In [10]:
%load_ext rpy2.ipython

/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning messages:

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 1: package ‘methods’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 2: package ‘datasets’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 3: package ‘utils’ was built under R version 3.3.3 

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 4: package ‘grDevic

In [11]:
%%R
# install.packages("psych")
# install.packages("matrixcalc")
# install.packages("corpcor")
# install.packages("Matrix")
# install.packages("glasso")
install.packages("qgraph")

/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Installing package into ‘/home/fede/R/x86_64-pc-linux-gnu-library/3.3’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: also installing the dependencies ‘nloptr’, ‘RcppEigen’, ‘lme4’, ‘arm’, ‘mi’, ‘rjson’, ‘whisker’, ‘ellipse’, ‘lavaan’, ‘sem’, ‘jpeg’, ‘png’, ‘sna’, ‘huge’, ‘fdrtool’, ‘d3Network’, ‘ggm’, ‘gtools’


  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://cran.stat.unipd.it/src/contrib/nloptr_1.0.4.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Content type 'application/octet-stream'
  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-package

/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://cran.stat.unipd.it/src/contrib/fdrtool_1.2.15.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning:  length 58387 bytes (57 KB)

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: downloaded 57 KB


  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: trying URL 'https://cran.stat.unipd.it/src/contrib/d3Network_0.5.2.1.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning:  length 22813 bytes (22 KB)

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntim

--- Please select a CRAN mirror for use in this session ---


In [12]:
%%R

library(psych)
library(matrixcalc)
library(Matrix)
library(glasso)
library(qgraph)
# E-step in the optimization algorithm:
Estep <- function(
  S, # Sample covariance
  Kcur, # Current estimate for K
  obs # Logical indicating observed
)
{
  stopifnot(require("Matrix"))
  stopifnot(isSymmetric(S))
  if (missing(Kcur))
  {
    if (missing(obs)) 
    {
      Kcur <- diag(nrow(S)) 
    } else 
    {
      Kcur <- diag(length(obs))
    }
  }
  stopifnot(isSymmetric(Kcur))
  if (missing(obs)) obs <- 1:nrow(Kcur) %in% 1:nrow(S)
  
  # To make life easier:
  H <- !obs
  O <- obs
  
  
  # Current estimate of S:
  library(corpcor)
  Scur <- pseudoinverse(Kcur)
  
  # Expected Sigma_OH:
  Sigma_OH <- S %*% pseudoinverse(Scur[O,O]) %*% Scur[O, H]
  #   Sigma_OH
  
  # Expected Sigma_H:
  Sigma_H <- Scur[H, H] - Scur[H,O] %*% pseudoinverse(Scur[O,O]) %*% Scur[O,H] + Scur[H,O] %*% pseudoinverse(Scur[O,O]) %*% S %*% pseudoinverse(Scur[O,O]) %*% Scur[O, H]
  
  # Construct expected sigma:
  Sigma_Exp <- rbind(cbind(S,Sigma_OH),cbind(t(Sigma_OH), Sigma_H))  
  
  return(Sigma_Exp)
}

# M-step in the optimization algorithm:
Mstep <- function(
  Sexp, # Expected full S
  obs, # Logica indiating oberved variables
  rho = 0,
  lambda
)
{
  if (!is.positive.definite(Sexp))
  {
    Sexp <- as.matrix(nearPD(Sexp)$mat)
    warning("Expected covariance matrix is not positive definite")
  }
  # Rho matrix:
  n <- nrow(Sexp)
  RhoMat <- matrix(rho, n, n)
  RhoMat[!obs,] <- 0
  RhoMat[,!obs] <- 0
  
  # Lambda:
  zeroes <- which(!lambda, arr.ind = TRUE)
  zeroes[,2] <- which(!obs)[zeroes[,2]]
  
  if (nrow(zeroes) > 0){
    K <- glasso(Sexp, RhoMat, penalize.diagonal=FALSE, zero = zeroes)$wi 
  } else {
    K <- glasso(Sexp, RhoMat, penalize.diagonal=FALSE)$wi
  }
  
  return(K)  
}

### Main lvglasso function
lvglasso <- function(
  S, # Sample cov
  nLatents, # Number of latents
  rho = 0, # Penalty
  thr = 1.0e-4, # Threshold for convergence (sum absolute diff)
  maxit = 1e4, # Maximum number of iterations
  lambda, # Logical matrix indicating the free factor loadings. Defaults to full TRUE matrix.
  ... # qgraph arguments
)
{
  if (missing(nLatents)){
    stop("'nLatents' must be specified")
  }
  
  nobs <- nrow(S)
  ntot <- nobs + nLatents
  
  if (missing(lambda) || is.null(lambda)){
    lambda <- matrix(TRUE, nobs, nLatents)
  }
  
  if (nrow(lambda) != nobs | ncol(lambda) != nLatents) stop("Dimensions of 'lambda' are wrong.")
  
  # PCA prior for K:
  efaRes <- principal(S, nfactors = nLatents)
  
#   # If sampleSizeis missing, set to 1000. Is only used for prior anyway.
# #   if (missing(sampleSize)){
#     sampleSize <- 1000
# #   }
# 
#   #   Get prior for K:
#   efaRes <- fa(S, n.obs= sampleSize, nfactors=nLatents)
  resid <- residuals(efaRes)
  class(resid) <- "matrix"
  load <- loadings(efaRes)
  class(load) <- "matrix"
  r <- efaRes$r.scores
  class(r) <- "matrix"
  r <- diag(diag(r))
#   
#   # Stupid nonanalytic way to get prior:
#   # Simulate N random variales:
#   #   eta <- rmvnorm(10000, rep(0, nLatents), r)
#   # 
#   #   # Simulate oserved scores:
#   #   Y <- eta %*% t(load) + rmvnorm(10000, rep(0, nobs), diag(diag(resid)
  # RAM FRAMEWORK:
  
  Sym <- rbind(cbind(diag( efaRes$uniquenesses),matrix(0,nobs,nLatents) ),cbind(t(matrix(0,nobs,nLatents) ),r))
  As <- matrix(0, ntot, ntot)
  if (nLatents > 0) As[1:nobs, (nobs+1):ntot] <- load
  
  Sigma <- solve(diag(ntot) - As) %*% Sym %*% t(solve(diag(ntot) - As))
  
  # Compute K:
# browser()
  K <- solve(cor2cov(cov2cor(Sigma),c(sqrt(diag(S)), rep(1, nLatents) )))
  #  K <- K  
#   K <- cov2cor(K)
      K=round(K,10)
  if (!is.positive.definite(K))
  {
    K <- as.matrix(nearPD(K)$mat)
#     warning("Expected covariance matrix is not positive definite")
  }

# browser()

  #   K <- matrix(-0.5,ntot,ntot)
  #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  
  #   K <- round(K,2)
  
  #   K <- EBICglasso(cov(cbind(Y,eta)), sampleSize)
  #   K <- as.matrix(forceSymmetric(cbind(rbind(pseudoinverse(resid),t(-load)), rbind(-load,pseudoinverse(r)))))
  
  # K <- as.matrix(forceSymmetric(cbind(rbind(pseudoinverse(resid),-0.5), rbind(rep(-0.5,nobs),1))))
  #   K <- as.matrix(forceSymmetric(cbind(rbind(diag(nrow(S)),t(-load)), rbind(-load,pseudoinverse(r)))))
  # browser()
  #   K <- matrix(-0.5,ntot,ntot)
  #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  #   obs <- c(rep(TRUE,nrow(S)), rep(FALSE,nLatents))
  # 
  #   K <- rbind(cbind(2*diag(nobs),-load),cbind(t(-load), 2*diag(ntot - nobs)))
  # rownames(K) <- colnames(K) <- NULL
  # 
  # diag(K) <- diag(K) - min(eigen(pseudoinverse(K))$values)
  
  # #   K[1:nobs,1:nobs] <- 0
  #   diag(K) <- 1
  obs <- c(rep(TRUE,nrow(S)), rep(FALSE,nLatents))
# 
# # Stupid prior:
# K <- matrix(0, ntot, ntot)
# diag(K) <- 1
# if (nLatents > 0) K[1:nobs, (nobs+1):ntot] <- K[ (nobs+1):ntot, 1:nobs] <- -1/nobs
  
  #   is.positive.definite(as.matrix(forceSymmetric(Estep(S, K, obs))))
  ### EM ###
it <- 1
Kold <- K
  repeat
  {
    Sexp <- as.matrix(forceSymmetric(Estep(S, K, obs)))
    #     Sexp <- cor2cov(cov2cor(Sexp),ifelse(obs,sqrt(diag(Sexp)),1))
    K <- as.matrix(forceSymmetric(Mstep(Sexp, obs, rho, lambda)))
    #     qgraph(wi2net(K), layout = "spring")
    
    # Check for convergence:
    if (sum(abs(cov2cor(pseudoinverse(Kold)[obs,obs]) - cov2cor(pseudoinverse(K)[obs,obs]))) < thr){
      break
    } else {
      it <- it + 1
      if (it > maxit){
        warning("Algorithm did not converge!")
        break
      } else {
        Kold <- K
      }
    }
  }

  if (!is.null(colnames(S)))
  {
    colnames(K) <- c(colnames(S),rep(paste0("F",seq_len(nLatents)))) 
  }
  
  if (is.null(rownames(S)))
  {
#       print(c(rep(paste0("N",seq_len(dim(K)[1] - nLatents))),rep(paste0("F",seq_len(nLatents)))) )
    rownames(K) <-c(rep(paste0("N",seq_len(dim(K)[1] - nLatents))),rep(paste0("F",seq_len(nLatents)))) 
  }

  # Partial correlations:
  pc <- qgraph:::wi2net(K)
  diag(pc) <- 1
  
rownames(pc) <- colnames(pc) <- rownames(K) <- colnames(K)

# Compute psychometric matrices:
Theta <- solve(K[obs, obs])
Lambda <- -Theta %*% K[obs,!obs]
Psi <- solve(K[!obs, !obs] - t(Lambda) %*% K[obs, obs] %*% Lambda)

# Return list mimics glasso:
Res <- list(
  w = pseudoinverse(K), # Estimated covariance matrix
  wi = K, # Estimated precision matrix
  pcor = pc, # Estimated partial correlation matrix
  observed = obs, # observed and latents indicator
  niter = it, # Number of iterations used in the algorithm
  lambda = Lambda,
  theta = Theta,
  psi = Psi
    )

  class(Res) <- "lvglasso"

  return(Res)
}


Error in library(qgraph) : there is no package called ‘qgraph’


/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in library(qgraph) : there is no package called ‘qgraph’

  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/home/fede/miniconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: There were 12 warnings (use warnings() to see them)
  warnings.warn(x, RRuntimeWarning)


In [44]:
from sklearn.covariance import empirical_covariance

covariance = empirical_covariance(X)

In [13]:
%%R -i covariance -o Res

Res <- lvglasso(covariance, 3)

NameError: name 'covariance' is not defined